In [ ]:
from pyomo.environ import *
import pandas as pd
import functions

#### Parameters

In [ ]:
# Set the value of 'beta' to 0.3 and 'alfa' to 0.5
beta = 0.3
alfa = 0.5

# Read a Parquet file named 'vehicles.parquet' and store it in the DataFrame 'df'
df = pd.read_parquet("files\\vehicles.parquet")
# Calculate the total number of unique 'ID' values in the 'df' DataFrame
total_vehicles = df["ID"].nunique()

# Read a Parquet file named 'antennas.parquet' and store it in the DataFrame 'df_RSU'
df_RSU = pd.read_parquet("files\\antennas.parquet")
# Calculate the total number of unique 'IDR' values in the 'df_RSU' DataFrame
total_RSU = df_RSU["IDR"].nunique()

# Load a pickled file named 'permanence_dict.pkl' and store its content in the 'permanence' variable
permanence = functions.load_pickle("files\\permanence_dict.pkl")

#### Model

In [ ]:
def constraint_0(model):
    """
    """
    global alfa
    
    return sum(model.qos[v] for v in model.Vehicles) / (total_vehicles) >= beta



def constraint_1(model, v):
    """
    """
    global beta, permanence, df

    return sum(model.on[i] * permanence[(v, i)]  for i in model.Antenna if (v, i) in permanence) >= alfa * model.qos[v]



# Objective Function : Minimize 
def function_objective(model):

    obj = sum(   model.on[i]
                    for i in model.Antenna )
    
    return obj 

In [ ]:
# Creating the Model
model = ConcreteModel()

# Creating the Sets
model.Antenna = RangeSet(0, total_RSU-1) # Set of RSU
model.Vehicles = RangeSet(0, total_vehicles-1) # Set of Vehicles

# Creating Variables
    
# on[i] = 1, if antenna c is on
model.on = Var(  model.Antenna,
                within = Binary,
                initialize = 0 
            )

# qos[v] = 1, if vehicle v in connected
model.qos = Var(model.Vehicles,
                within = Binary,
                initialize = 0 
            )


model.objective = Objective(    rule = function_objective, 
                                sense = minimize
                            )

model.C0 = Constraint(  rule = constraint_0 )
model.C1 = Constraint(  model.Vehicles,
                        rule = constraint_1 )

In [ ]:
# Selecting and Creating the Solver
# ----------------------------
solver = SolverFactory("cplex")
solver.options["timelimit"] = 10000

# Calling the solver and solving the model
# ----------------------------

results = solver.solve( model, 
                        #tee = True, 
                        #keepfiles = True, 
                    )

# Print the objective function value from the 'model' and display it as part of a message
print(f"FO value: {value(model.objective)}")

# Create a list 'all_results' containing 'results' and 'model' and save it to a pickle file named 'Solution_model.pkl'
all_results = [results, model]
functions.save_pickle(results, "files\\Solution_model.pkl")

In [ ]:
# Create an empty list 'on_off' to store the values of 'model.on[i]' for each 'i'
on_off = []

# Loop through a range of values from 0 to 'total_RSU'
for i in range(total_RSU):

    # Append the value of 'model.on[i]' to the 'on_off' list
    on_off.append(value(model.on[i]))

# Add a new column 'ON' to the 'df_RSU' DataFrame and populate it with the values from the 'on_off' list
df_RSU["ON"] = on_off
# Save the modified 'df_RSU' DataFrame to a pickle file named 'Solution_PL.pkl'
functions.save_pickle(    df_RSU, 
                "files\\Solution_PL.pkl")